Setting up the Notebook

In [1]:
!pip install -q langchain langchain-openai llama-index pyautogen crewai openai llama-index-embeddings-huggingface langchain-community

In [2]:
!pip list | grep -E "langchain|llama-index|pyautogen|crewai|openai"

crewai                                   1.6.1
langchain                                1.2.0
langchain-classic                        1.0.1
langchain-community                      0.4.1
langchain-core                           1.2.5
langchain-openai                         1.1.6
langchain-text-splitters                 1.1.0
llama-index                              0.14.10
llama-index-cli                          0.5.3
llama-index-core                         0.14.10
llama-index-embeddings-huggingface       0.6.1
llama-index-embeddings-openai            0.5.1
llama-index-indices-managed-llama-cloud  0.9.4
llama-index-instrumentation              0.4.2
llama-index-llms-openai                  0.6.12
llama-index-readers-file                 0.5.6
llama-index-readers-llama-parse          0.5.1
llama-index-workflows                    2.11.6
openai                                   2.12.0
pyautogen                                0.10.0


In [3]:
!pip install --upgrade jedi

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "secret-key"  # Replace with real key

LlamaIndex RAG flow

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["agenticai.pdf"]).load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [6]:
response = query_engine.query("Summarize the key lessons in this Agentic AI document")
print(response)

The key lessons in this Agentic AI document emphasize the importance of humility in the field of building AI agents. It highlights the idea that in a constantly evolving field, it is crucial to maintain a humble attitude as perpetual beginners to create enduring solutions.


LangChain Agent Chain

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Create LlamaIndex query engine (already done in Phase 1, but ensure it's here)
query_engine = index.as_query_engine()

# Wrap query_engine.query as a LangChain runnable (takes string input, returns string response)
llama_retriever = RunnableLambda(lambda q: str(query_engine.query(q)))

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Prompt template
prompt = ChatPromptTemplate.from_template(
    "Using only this retrieved context from my resume:\n{context}\n\nAnswer the question: {question}"
)

# Build the chain
chain = (
    {"context": llama_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
result = chain.invoke("What are the top 3 learnings from the book AGENTIC AI?")
print(result)

The top 3 learnings from the book AGENTIC AI are:

1. The importance of breaking down problems into specific tools.
2. The value of structuring data for reasoning.
3. The significance of enabling agents to intelligently analyze information for providing recommendations.


CrewAI Multi-agent collaboration + AutoGen style

In [9]:
from crewai import Agent, Task, Crew

# Agent 1: Researcher
researcher = Agent(
    role="AI Trends Researcher",
    goal="Find and list top 5 AI agent trends for 2026",
    backstory="You are an expert in generative AI and agentic systems.",
    llm=ChatOpenAI(model="gpt-4o-mini"),
    allow_delegation=False,
    verbose=True  # Optional
)

# Agent 2: Summarizer
summarizer = Agent(
    role="Summary Writer",
    goal="Condense research into clear bullet points",
    backstory="You are a concise technical writer.",
    llm=ChatOpenAI(model="gpt-4o-mini"),
    allow_delegation=False,
    verbose=True
)

# Tasks
task1 = Task(
    description="Research and list the top 5 emerging AI agent trends for 2026.",
    expected_output="A numbered list of 5 trends with a one-sentence explanation each.",
    agent=researcher
)

task2 = Task(
    description="Take the research output and summarize it into 4-5 concise bullet points.",
    expected_output="Bullet point summary highlighting the key insights.",
    agent=summarizer
)

# Create the Crew – FIX: verbose must be True or False
crew = Crew(
    agents=[researcher, summarizer],
    tasks=[task1, task2],
    verbose=True   # ← Changed from 2 to True
)

# Run it
result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 60d6e942-c7c1-4b2b-af33-8a5558a35a66                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI Trends Researcher                                                                                    │
│                                                                                                                 │
│  Task: Research and list the top 5 emerging AI agent trends for 2026.                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

Output()

Simple Autonomous Agent Simulation (replaces Auto-GPT)

In [10]:
!pip install -q duckduckgo-search langchain-community

In [14]:
!pip install -q langgraph

In [16]:
!pip install -q langgraph ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 8.7 MB/s eta 0:00:00


In [17]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent  # Official modern ReAct agent

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Search tool (now works after ddgs install)
search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

# Create the modern ReAct agent using LangGraph
agent = create_react_agent(llm, tools)

# Input query
inputs = {
    "messages": [("user", "Analyze AI job market trends as of late 2025. What are the most in-demand skills for roles building AI agents, multi-agent systems, RAG, Copilot extensibility, and agentic frameworks? Include sources if possible.")]
}

# Stream the reasoning steps
print("=== AUTONOMOUS AGENT REASONING ===\n")
for chunk in agent.stream(inputs, stream_mode="values"):
    messages = chunk.get("messages", [])
    if messages:
        last_message = messages[-1]
        if hasattr(last_message, "content") and last_message.content:
            print(last_message.content)
        if hasattr(last_message, "tool_calls") and last_message.tool_calls:
            print(f"Tool Call: {last_message.tool_calls}")
        print("---")

print("\n=== FINAL ANSWER PRINTED ABOVE ===")

/tmp/ipython-input-2675095721.py:13: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


=== AUTONOMOUS AGENT REASONING ===

Analyze AI job market trends as of late 2025. What are the most in-demand skills for roles building AI agents, multi-agent systems, RAG, Copilot extensibility, and agentic frameworks? Include sources if possible.
---
Tool Call: [{'name': 'duckduckgo_search', 'args': {'query': 'AI job market trends 2025 in-demand skills for AI agents multi-agent systems RAG Copilot extensibility agentic frameworks'}, 'id': 'call_Or6ci2hjxymS0ysIvkJ3xQ8j', 'type': 'tool_call'}]
---
The agentic AI market represents more than just technological advancement — it’s a fundamental shift in how businesses operate and compete. Organizations that understand this transformation and act decisively will capture disproportionate value in the coming decade. 1. AI Agents and Agentic AI in Python: Powered by Generative AI - Vanderbilt University.Your Next Steps in the Agentic AI Revolution. 2025 represents a pivotal moment in AI development, with autonomous agents transitioning from e